In [17]:

#Import Libraries
import pandas as pd
import numpy as np
import xarray as xr
import os
import cv2
import matplotlib
from scipy.ndimage import gaussian_filter
import datetime
from datetime import date, timedelta
import wget
from math import floor
from pydap.client import open_url
import netCDF4 as nc
import matplotlib.pyplot as plt

matplotlib.use('Agg')    #por causa do erro AttributeError: 'NoneType' object has no attribute 'set_cursor'
%matplotlib inline

In [208]:
def lcm(*args):
    
    count = 0   #para contar quantos args existem
    res = []    #list of lists com os resultados. Cada list tem os multiplos desse arg
    
    for arg in args:
        aux = []     #list com os multiplos para cada arg
        if arg == None:
            return 1
        if arg == 0:
            return 0
        for i in range(1,10):
            aux.append(arg*i)
        res.append(aux)
    

In [259]:
args = (2,3,4)
count = 0   #para contar quantos args existem
res = []    #list of lists com os resultados. Cada list tem os multiplos desse arg

for arg in args:
    aux = []
    if arg == None:
        print(1)
    if arg == 0:
        print(0)
    for i in range(1,10):
        aux.append(arg*i)
    res.append(aux)
    count += 1
    
while count > 0:
    for elem in res[0]:
        if (elem in res[count-1]) & (elem in res[2]):
            print(elem)
    

12


In [261]:
len


3

In [207]:
li[0][2] in li[2]

True